In [ ]:
#conda install -c bioconda mafft 
#!pip install Biopython

Here I convert a SeqIO object to a string then to bytes. I then pass this encoded string to a subprocess call of maft through STDOUT. Mafft reads the encoded fasta through STDIN and ouputs the aligned fasta through STDOUT. This STDOUT is then decoded back into a python string and read as a new aligned SeqIO object.

I have supplied a function at the bottom that takes unaligned SeqIO objects and returns alinged SeqIO objects

In [1]:
from Bio import SeqIO
import subprocess
from io import StringIO

In [2]:
!ls

alignment.ipynb  sars_map.ipynb  test_500_seqs.fasta


In [3]:
seqs = list(SeqIO.parse('test_500_seqs.fasta', 'fasta'))

In [5]:
len(seqs)

500

In [6]:
#This is just a command line example of what is happening in the python code below
!cat test_500_seqs.fasta | mafft --quiet - > aligned_file.fasta

In [7]:
!ls

aligned_file.fasta  alignment.ipynb  sars_map.ipynb  test_500_seqs.fasta


In [8]:
seq_str = ''
for seq in seqs:
    seq_str += '>' + seq.description + '\n'
    seq_str += str(seq.seq) + '\n'

In [10]:
child = subprocess.Popen(['mafft', '--quiet', '-'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
child.stdin.write(seq_str.encode())
child_out = child.communicate()[0].decode('utf8')
seq_ali = list(SeqIO.parse(StringIO(child_out), 'fasta'))
child.stdin.close()

In [14]:
#to write to file
with open('aligned_file.fasta', 'w') as f:
    for seq in seqs:
        f.write( '>' + seq.description + '\n')
        f.write(str(seq.seq) + '\n')


In [ ]:
#in a function. takes unaligned SeqIO object and returns aligned SeqIO object
def align_seqs(seqs):
    seq_str = ''
    for seq in seqs:
        seq_str += '>' + seq.description + '\n'
        seq_str += str(seq.seq) + '\n'
    child = subprocess.Popen(['mafft', '--quiet', '-'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    child.stdin.write(seq_str.encode())
    child_out = child.communicate()[0].decode('utf8')
    seq_ali = list(SeqIO.parse(StringIO(child_out), 'fasta'))
    child.stdin.close()
    return seq_ali